#### Importar librarias

In [5]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import yt_dlp
import pandas as pd
import re
import os

#### Spotify URL playlist & credentials

In [6]:
# URL de la lista
url_playlist = "https://open.spotify.com/playlist/37i9dQZF1EIgrEQA5FONTg"
# https://open.spotify.com/playlist/37i9dQZF1EIgrEQA5FONTg melancholy
# https://open.spotify.com/playlist/1v68Iqgxoagm8pYzphL7Qv melancholy 2


In [7]:
# Configuración de credenciales de cliente de Spotify
client_id = 'YOUR CLIENT ID'
client_secret = 'YOUR CLIENT PASSWORD'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

#### Obtain playlist's uris
To allow 429 error response from Spotify (reached their Web API rate limit), different functions with loop will be created for each query.

In [8]:
def obtener_uris_playlist(playlist_id):
    uris = []
    offset = 0
    # Iterar sobre cada una de las páginas (items) de la lista de reproducción
    while True:
        results = sp.playlist_tracks(playlist_id, offset=offset)
        items = results.get('items', [])
        if not items:
            break
        for item in items:
            track_info = item.get('track')
            if track_info:
                uri = track_info.get('uri')
                uris.append(uri)
        offset += len(items)
    return uris


#### Obtain str Track - Srtist list

In [9]:
def obtener_tracks_playlist(playlist_id):
    tracks = []
    offset = 0
    while True:
        results = sp.playlist_tracks(playlist_id, offset=offset)
        items = results.get('items', [])
        if not items:
            break
        for item in items:
            track_info = item.get('track')
            if track_info:
                track = {
                    "nombre": track_info.get('name', ''),
                    "artista": track_info.get('artists', [{}])[0].get('name', '')
                }
                tracks.append(track)
        offset += len(items)
    return tracks

#### Obtain track features

In [10]:
def obtener_caracteristicas(uris):
    features = []
    for uri in uris:
        try:
            caracteristicas = sp.audio_features(uri)[0]
            if caracteristicas:
                features.append(caracteristicas)
        except Exception as e:
            print(f"Error al obtener características de {uri}: {e}")
    return features


In [11]:
# Obtener la lista de canciones de la playlist
playlist_id = re.search(r'playlist\/(\w{22})', url_playlist).group(1)
uris = obtener_uris_playlist(playlist_id)
tracks = obtener_tracks_playlist(playlist_id)
features = obtener_caracteristicas(uris)

In [12]:
# Remove tracks with no features
uris_no_encontrados = [uri for uri in uris if not any(feature["uri"] == uri for feature in features)]
for uri in uris_no_encontrados:
    print(f"La URI {uri} no está en la lista de features. Posición {uris.index(uri)}")
if uris_no_encontrados:
    indice = uris.index(uri)
    del uris[indice]
    del tracks[indice]

In [13]:
df1 = pd.DataFrame(tracks)
df2 = pd.DataFrame(features)
df = pd.concat([df1, df2], axis=1)

In [14]:
columnas_interes = ["nombre", "artista", "acousticness", "danceability", "duration_ms", "energy", "instrumentalness", 
                    "liveness", "loudness", "speechiness", "tempo", "valence", "key", 
                    "time_signature", "mode"]
df_features = df[columnas_interes]
df_features


,nombre,artista,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,key,time_signature,mode
0,M.A.I,Milo j,0.758000,0.364,184027,0.5720,0.000000,0.2290,-5.861,0.0341,80.363,0.514,10,4,0
1,La Última Canción,Rels B,0.426000,0.793,184749,0.6050,0.000000,0.3340,-4.952,0.1190,91.068,0.268,1,4,0
2,S.A.D,KHEA,0.760000,0.862,128375,0.3060,0.000000,0.0946,-13.580,0.0499,120.005,0.772,8,4,1
3,Ya No Te Hago Falta,Sen Senra,0.934000,0.560,161600,0.3110,0.000060,0.0706,-8.508,0.0380,180.050,0.452,9,4,1
4,HACIENDO QUE ME AMAS,Bad Bunny,0.869000,0.584,217769,0.4110,0.000045,0.0955,-7.147,0.0838,169.934,0.412,10,4,0
5,Cantándote al Oído,KYR4,0.915000,0.735,165600,0.3270,0.000070,0.1710,-7.844,0.0487,99.907,0.661,4,4,0
6,Ayer Me Acordé de Ti,Enol,0.336000,0.756,176500,0.6420,0.000000,0.1230,-6.297,0.0749,90.032,0.586,7,4,1
7,Porque Te Vas (Jedet Version),Cecilio G.,0.458000,0.553,205896,0.1760,0.000000,0.1100,-15.096,0.0642,89.237,0.538,4,4,0
8,San Blas,Pole.,0.530000,0.443,161280,0.6620,0.000000,0.3140,-5.745,0.0471,92.859,0.675,4,4,1
9,Veneno (Acústico),DELLAFUENTE,0.874000,0.393,202687,0.2760,0.000003,0.1190,-8.307,0.0798,78.474,0.245,7,3,0


In [15]:
# Guardar las características en un archivo CSV
save_path = r'C:\Users\Fabian\Desktop\PROYECTO\canciones\melancholy\Spotify_features_FABI.csv'
df_features.to_csv(save_path, index=False)

print("Extracción de características musicales completa.")

Extracción de características musicales completa.



En resumen, las características como 
- acousticness alto: podría relacionarse con un sonido más íntimo y menos producido
- baja danceability: baja danceability, ya que suelen carecer de un ritmo animado o enérgico.
- baja energía: refleja una atmósfera más calmada o introspectiva
- instrumentalidad: la falta de letras vocales puede permitir que la música evite distracciones emocionales
- loudness bajo: puede contribuir a una sensación de intimidad y reflexión.
- tempo lento: Las canciones tristes tienden a tener un tempo más lento
- valencia baja: suelen asociarse con sentimientos de tristeza, nostalgia o melancolía.

son indicadores potenciales de música triste o melancólica